# Analisis de Imagenes para texto alternativo

In [1]:
import json
import boto3
from IPython.display import display, HTML
import base64

### 1. Crea el cliente Bedrock usando SDK de AWS

In [2]:
modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
accept = 'application/json'
contentType = 'application/json'

bedrock = boto3.client("bedrock-runtime")


### 3. Analiza la imagen y crea el texto alternativo

In [12]:
image_path = "./samples/sample_diagram.jpg"

with open(image_path, "rb") as image_file:
    content_image = base64.b64encode(image_file.read()).decode('utf8')


message = {
    "role": "user",
    "content": [
        {
            "type": "image", 
            "source": {
                "type": "base64", 
                "media_type": "image/jpeg", 
                "data": content_image
            }
        }
    ]
}

system_prompt = "Tu eres un revisor de articulos web que van a ser publicados, tu misión es ver las imágenes y proporcione un texto alternativo (que se incluirá como atributo 'alt' para la etiqueta img) que describa su contenido. Responde en 150 caracteres o menos sin preámbulo"

body = {
    "system": system_prompt,
    "messages":[message],"anthropic_version":"bedrock-2023-05-31","max_tokens":75, "temperature":0
}

response = bedrock.invoke_model(body=json.dumps(body), modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

alt_text = response_body.get("content")[0].get("text")

### 4. Resultado

In [14]:

# Imagen con alt y src
display(HTML(f"<strong>Texto Alternativo:</strong><br/>{alt_text}<br/><img alt='{alt_text}' src='{image_path}'>"))
